## Import Libs

In [30]:
import os
import requests
import pandas as pd
from pandas.io.json import json_normalize
import json
from PyPDF2 import PdfFileWriter, PdfFileReader
import time


## Setup Endpoints and Files

In [38]:
# API details
APIEndpoint = "" #Your endpoint here "https://[YourEndpoint].cognitiveservices.azure.com/formrecognizer/documentModels/"
APIKey = "" # APIKEY
Model = ""   #Your trained model

# the target PDF document to split
filename = r''

# the Excel file to export inferences
OutputExcel = r''

print(filename)

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\Catalogo 29-1.16.2023.pdf


## Splif PDF File

Create multi files for every page on the original document.  Add all files to a python list.  Will later be used to iterate it thru form recognizer

In [39]:
fileList = list()

inputpdf = PdfFileReader(open(filename, "rb"))

for i in range(inputpdf.numPages):
    output = PdfFileWriter()
    output.addPage(inputpdf.getPage(i))
    with open("document-page%s.pdf" % i, "wb") as outputStream:
        output.write(outputStream)
        fileList.append(r'' % i) #Destination Directory for your split files

In [40]:
print(fileList)
len(fileList)

['C:\\Users\\hcquinta\\OneDrive - Microsoft\\Documents 1\\My MSFT\\[personal]\\chasing\\source\\split\\document-page0.pdf', 'C:\\Users\\hcquinta\\OneDrive - Microsoft\\Documents 1\\My MSFT\\[personal]\\chasing\\source\\split\\document-page1.pdf', 'C:\\Users\\hcquinta\\OneDrive - Microsoft\\Documents 1\\My MSFT\\[personal]\\chasing\\source\\split\\document-page2.pdf', 'C:\\Users\\hcquinta\\OneDrive - Microsoft\\Documents 1\\My MSFT\\[personal]\\chasing\\source\\split\\document-page3.pdf', 'C:\\Users\\hcquinta\\OneDrive - Microsoft\\Documents 1\\My MSFT\\[personal]\\chasing\\source\\split\\document-page4.pdf', 'C:\\Users\\hcquinta\\OneDrive - Microsoft\\Documents 1\\My MSFT\\[personal]\\chasing\\source\\split\\document-page5.pdf', 'C:\\Users\\hcquinta\\OneDrive - Microsoft\\Documents 1\\My MSFT\\[personal]\\chasing\\source\\split\\document-page6.pdf', 'C:\\Users\\hcquinta\\OneDrive - Microsoft\\Documents 1\\My MSFT\\[personal]\\chasing\\source\\split\\document-page7.pdf', 'C:\\Users\\hcq

45

# Request Analysis

Analysis is done async, we need two API calls, 
1. To request the analysis
2. To request the results of the analysis

- Setup the cognitive service and send PDF file for analysis
- Get the request Id sent back by the service located in the headers (APIM Request Id)

In [41]:
def Inference(FileName):
    url = APIEndpoint + Model + ":analyze?api-version=2022-08-31"
    payload =  {'file': open(r'%s' %FileName, 'rb')}
    headers = {
    'Ocp-Apim-Subscription-Key': APIKey,
    'Content-Type': 'application/pdf'
    }
    
    response = requests.request("POST", url, headers=headers, files=payload)
    print (response.text)
    return response.headers.get('apim-request-id')


In [42]:

def GetInference(request_id):
  # if response.status_code == 200 or response.status_code == 202:
  time.sleep(10)
  
  nTries = 20
  nTry = 0    

  while nTry < nTries:
    try:
      url = APIEndpoint + Model + "/analyzeResults/" + request_id + "?api-version=2022-08-31"
      payload = "{}"
      headers = {
        'Ocp-Apim-Subscription-Key': APIKey,
        'Content-Type': 'application/pdf'
      }  

      response = requests.request("GET", url, headers=headers, data=payload)
      if json.loads(response.text)['status'] == "succeeded":
        json_data = json.loads(response.text)['analyzeResult']['documents']
      
        for item in json_data:
          df = pd.DataFrame(item['fields'])
      
        df['style'] = df['style'].replace(to_replace= r'\n', value= ' ', regex=True)
        df.loc[['content'],  ['brewery', 'beer', 'style', 'abv', 'untappd', 'price']]
        return df

      time.sleep(3)  
      nTry += 1
    except Exception as e:
        return  "error"




## Execute and Results

Skip the first file, subindex[0] since that's always an empty page.  
1. Request Inference
2. Go back for the result
3. Append result in a Dataframe
4. Export to Excel File 

In [43]:
ExportResults = pd.DataFrame()

for i in range(len(fileList)):
    if i != 0:
        print(fileList[i])
        request_id = Inference(fileList[i])
        InferenceResult = GetInference(request_id)
        ExportResults = ExportResults.append(InferenceResult)

ExportResults.loc[['content'],  ['brewery', 'beer', 'style', 'abv', 'untappd', 'price']].to_excel(r'', header=True) #Destination file for your exported results

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\split\document-page1.pdf

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\split\document-page2.pdf

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\split\document-page3.pdf

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\split\document-page4.pdf

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\split\document-page5.pdf

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\split\document-page6.pdf

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\split\document-page7.pdf

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\split\document-page8.pdf

C:\Users\hcquinta\OneDrive - Microsoft\Documents 1\My MSFT\[personal]\chasing\source\split\document-page9.pdf

C